In [36]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
from sklearn.utils.class_weight import compute_class_weight
import os
import numpy as np

In [ ]:
train_dir = r"C:\Users\PMLS\Downloads\datasets\augmentation\Dataset\train"  
class_names = sorted(os.listdir(train_dir))  
num_classes = len(class_names)
print("Detected Classes in Directory:")
print(class_names)

Detected Classes in Directory:
['Acne and Rosacea Photos', 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions', 'Atopic Dermatitis Photos', 'Cellulitis Impetigo and other Bacterial Infections', 'Eczema Photos', 'Exanthems and Drug Eruptions', 'Herpes HPV and other STDs Photos', 'Light Diseases and Disorders of Pigmentation', 'Lupus and other Connective Tissue diseases', 'Melanoma Skin Cancer Nevi and Moles', 'Poison Ivy Photos and other Contact Dermatitis', 'Psoriasis pictures Lichen Planus and related diseases', 'Seborrheic Keratoses and other Benign Tumors', 'Systemic Disease', 'Tinea Ringworm Candidiasis and other Fungal Infections', 'Urticaria Hives', 'Vascular Tumors', 'Vasculitis Photos', 'Warts Molluscum and other Viral Infections']


In [ ]:
class_indices = {class_name: idx for idx, class_name in enumerate(class_names)}
print("\nCorrected Class Indices Mapping:")
print(class_indices)



Corrected Class Indices Mapping:
{'Acne and Rosacea Photos': 0, 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions': 1, 'Atopic Dermatitis Photos': 2, 'Cellulitis Impetigo and other Bacterial Infections': 3, 'Eczema Photos': 4, 'Exanthems and Drug Eruptions': 5, 'Herpes HPV and other STDs Photos': 6, 'Light Diseases and Disorders of Pigmentation': 7, 'Lupus and other Connective Tissue diseases': 8, 'Melanoma Skin Cancer Nevi and Moles': 9, 'Poison Ivy Photos and other Contact Dermatitis': 10, 'Psoriasis pictures Lichen Planus and related diseases': 11, 'Seborrheic Keratoses and other Benign Tumors': 12, 'Systemic Disease': 13, 'Tinea Ringworm Candidiasis and other Fungal Infections': 14, 'Urticaria Hives': 15, 'Vascular Tumors': 16, 'Vasculitis Photos': 17, 'Warts Molluscum and other Viral Infections': 18}


In [ ]:
# ✅ Step 3: Data Augmentation & Generators
batch_size = 32
image_size = (224, 224)

train_datagen = ImageDataGenerator(rescale=1.0 / 255.0,validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="sparse",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="sparse",
    subset="validation"
)

Found 2092 images belonging to 19 classes.
Found 517 images belonging to 19 classes.


In [30]:
class_counts = {class_name: len(os.listdir(os.path.join(train_dir, class_name))) for class_name in class_names}
total_samples = sum(class_counts.values())

class_weights = {class_indices[class_name]: total_samples / count for class_name, count in class_counts.items()}

print("\nComputed Class Weights:")
print(class_weights)


Computed Class Weights:
{0: 3.105952380952381, 1: 8.102484472049689, 2: 26.09, 3: 40.13846153846154, 4: 74.54285714285714, 5: 130.45, 6: 65.225, 7: 8.725752508361204, 8: 28.988888888888887, 9: 108.70833333333333, 10: 42.08064516129032, 11: 42.08064516129032, 12: 32.6125, 13: 32.20987654320987, 14: 21.385245901639344, 15: 869.6666666666666, 16: 25.831683168316832, 17: 173.93333333333334, 18: 10.52016129032258}


In [31]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(num_classes, activation="softmax")
])

In [32]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [33]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    class_weight=class_weights
)

Epoch 1/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 85s 1s/step - accuracy: 0.0731 - loss: 262.5596 - val_accuracy: 0.0368 - val_loss: 2.9469
Epoch 2/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.1218 - loss: 55.6466 - val_accuracy: 0.0851 - val_loss: 2.9206
Epoch 3/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.2241 - loss: 49.4051 - val_accuracy: 0.0909 - val_loss: 2.8981
Epoch 4/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 84s 1s/step - accuracy: 0.3575 - loss: 35.5979 - val_accuracy: 0.1238 - val_loss: 2.9220
Epoch 5/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.5763 - loss: 18.8059 - val_accuracy: 0.1954 - val_loss: 2.8059
Epoch 6/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.7476 - loss: 11.6866 - val_accuracy: 0.1857 - val_loss: 3.1215
Epoch 7/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.8393 - loss: 7.1559 - val_accuracy: 0.2089 - val_loss: 3.2336
Epoch 8/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.8944 - loss: 5.1844 - val_accuracy: 0.2669 - va

In [35]:
test_loss, test_accuracy = model.evaluate(train_generator)
print(f"\n✅ Test Accuracy: {test_accuracy * 100:.2f}%")

66/66 ━━━━━━━━━━━━━━━━━━━━ 18s 266ms/step - accuracy: 0.9831 - loss: 0.2418

✅ Test Accuracy: 98.04%


In [41]:

# ✅ Save Model
model.save("cnn_model.h5")